In [7]:
# import 
import os
import json
import pandas as pd
import shutil
import zipfile

In [ ]:
# 🔹 1️⃣ 압축 해제 (ZIP 파일 전체)
zip_folder = "압축파일_폴더경로"  # ZIP 파일들이 있는 폴더
extract_folder = "압축해제_폴더경로"  # 압축 해제할 폴더
csv_file_path = "nutrition_data.csv"  # CSV 파일 경로

# ZIP 파일 목록 가져오기
zip_files = [f for f in os.listdir(zip_folder) if f.endswith(".zip")]

for zip_file in zip_files:
    zip_path = os.path.join(zip_folder, zip_file)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)  # 압축 해제
    print(f"✅ {zip_file} 압축 해제 완료!")

# 🔹 2️⃣ CSV 데이터 로드
df = pd.read_csv(csv_file_path)
df = df[['식품명', '당류(g)']]

# 🔹 3️⃣ JSON 파일 수정 (당류 추가)
json_files = [f for f in os.listdir(extract_folder) if f.endswith(".json")]

for json_file in json_files:
    json_path = os.path.join(extract_folder, json_file)

    with open(json_path, "r", encoding="utf-8") as f:
        json_data = json.load(f)

    # JSON에서 음식명 매칭 후 당류 추가
    for item in json_data:
        food_name = item["Name"]  # JSON 속 음식명
        matching_row = df[df["식품명"] == food_name]  # CSV에서 검색
        
        if not matching_row.empty:
            item["당류(g)"] = matching_row["당류(g)"].values[0]
        else:
            item["당류(g)"] = None

    # 수정된 JSON 저장 (덮어쓰기)
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)

    print(f"✅ {json_file} 업데이트 완료!")

# 🔹 4️⃣ 원본 JSON 삭제 후 CSV 변환
all_data = []

for json_file in json_files:
    json_path = os.path.join(extract_folder, json_file)
    
    with open(json_path, "r", encoding="utf-8") as f:
        json_data = json.load(f)
    
    all_data.extend(json_data)  # JSON 데이터 합치기
    os.remove(json_path)  # 원본 JSON 삭제

# CSV 파일 저장
csv_output_path = "updated_food_data.csv"
df_json = pd.DataFrame(all_data)
df_json.to_csv(csv_output_path, index=False, encoding="utf-8-sig")

print("✅ 모든 작업 완료! JSON 원본 삭제 및 CSV 변환 완료!")